# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,conduaga,8.8620,117.4893,79.86,82,5,6.29,PH,Mon Dec 11 10:58:25 2023
1,1,oamaru,-45.0784,170.9801,52.38,86,100,11.74,NZ,Mon Dec 11 10:58:25 2023
2,2,basco,20.4487,121.9702,79.72,78,14,12.77,PH,Mon Dec 11 10:58:25 2023
3,3,palikir - national government center,6.9248,158.1611,81.34,83,40,5.75,FM,Mon Dec 11 10:58:25 2023
4,4,howrah,22.5892,88.3103,68.00,68,0,3.44,IN,Mon Dec 11 10:58:26 2023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
   "Lng",
    "Lat",
    geo = True,
    global_extent= True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    scale=.5,
    alpha=0.5,
    size = "Humidity",
    color="Humidity",
    color_key='Category10',
    hover_cols="Humidity"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
    # A max temperature higher than 40 degrees but lower than 60
    # Wind speed less than 4.5 m/s
    # Zero cloudiness

ideal_weather_df = city_data_df[(city_data_df['Max Temp'] > 40) & (city_data_df['Max Temp'] <60)& (city_data_df['Wind Speed'] < 4.5)&(city_data_df['Cloudiness']==0)]   

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
104,104,qujing,25.4833,103.7833,47.80,71,0,3.44,CN,Mon Dec 11 10:58:41 2023
140,140,silver city,32.7701,-108.2803,51.31,23,0,0.00,US,Mon Dec 11 10:58:49 2023
269,269,mogok,22.9167,96.5000,55.72,94,0,2.19,MM,Mon Dec 11 10:59:06 2023
275,275,zahedan,29.4963,60.8629,51.71,35,0,0.00,IR,Mon Dec 11 10:59:07 2023
458,458,live oak,36.9836,-121.9805,55.33,65,0,3.44,US,Mon Dec 11 10:59:40 2023


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df=ideal_weather_df.copy()
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
104,104,qujing,25.4833,103.7833,47.80,71,0,3.44,CN,Mon Dec 11 10:58:41 2023,
140,140,silver city,32.7701,-108.2803,51.31,23,0,0.00,US,Mon Dec 11 10:58:49 2023,
269,269,mogok,22.9167,96.5000,55.72,94,0,2.19,MM,Mon Dec 11 10:59:06 2023,
275,275,zahedan,29.4963,60.8629,51.71,35,0,0.00,IR,Mon Dec 11 10:59:07 2023,
458,458,live oak,36.9836,-121.9805,55.33,65,0,3.44,US,Mon Dec 11 10:59:40 2023,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
qujing - nearest hotel: No hotel found
silver city - nearest hotel: Murray Hotel
mogok - nearest hotel: Mogok Hill
zahedan - nearest hotel: هتل صالح;بانک ملی
live oak - nearest hotel: Chaminade Resort & Spa


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
104,104,qujing,25.4833,103.7833,47.80,71,0,3.44,CN,Mon Dec 11 10:58:41 2023,No hotel found
140,140,silver city,32.7701,-108.2803,51.31,23,0,0.00,US,Mon Dec 11 10:58:49 2023,Murray Hotel
269,269,mogok,22.9167,96.5000,55.72,94,0,2.19,MM,Mon Dec 11 10:59:06 2023,Mogok Hill
275,275,zahedan,29.4963,60.8629,51.71,35,0,0.00,IR,Mon Dec 11 10:59:07 2023,هتل صالح;بانک ملی
458,458,live oak,36.9836,-121.9805,55.33,65,0,3.44,US,Mon Dec 11 10:59:40 2023,Chaminade Resort & Spa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
   "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale=1,
    alpha=0.5,
    size="Humidity",
    color="City",
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)